In [1]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [6]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [43]:
# Classe Network
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)  # número de neurônios em cada camada
        self.sizes = sizes

        # Inicialização dos pesos de forma mais adequada para o ReLu
        self.biases = [np.random.randn(y, 1) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights = [np.random.randn(y, x) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])]

        # Inicializa os termos para o Momentum
        self.velocity_weights = [np.zeros_like(w) for w in self.weights]
        self.velocity_biases = [np.zeros_like(b) for b in self.biases]

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = relu(np.dot(w, x) + b)  # net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, initial_eta, lambda_reg=0.0, momentum=0.0, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        # dataset de treino
        training_data = list(training_data)
        n = len(training_data)

        # dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        accuracies = []

        for j in range(epochs):
            random.shuffle(training_data)
            # técnica que realiza o treinamento por lotes
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, initial_eta, lambda_reg, momentum, n)

            if test_data:
                acc = self.evaluate(test_data)
                accuracies.append((acc * 100) / n_test)
                print("Epoch {} : {} / {} = {:.2f}%".format(j, acc, n_test, (acc * 100) / n_test))
            else:
                print("Epoch {} finalizada".format(j))

        if test_data:
            avg_accuracy = np.mean(accuracies)
            print("\nAcurácia média após {} épocas: {:.2f}%".format(epochs, avg_accuracy))
            return avg_accuracy  # Retorna a acurácia média para uso posterior

    def update_mini_batch(self, mini_batch, eta, lambda_reg, momentum, n):
        """Atualiza os pesos e limiares da rede aplicando
        a descida do gradiente usando backpropagation para um único mini lote.
        O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        # Inicializa matrizes para as derivadas dos pesos e vieses
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            # Resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            # Soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        # Aplicando regularização e atualizando pesos e limiares com momentum
        for i in range(len(self.weights)):
            # Regularização L2: adiciona o termo lambda_reg/n * w
            l2_term = (lambda_reg / len(mini_batch)) * self.weights[i]

            # Regularização L1: adiciona o termo lambda_reg/n * sign(w)
            l1_term = (lambda_reg / len(mini_batch)) * np.sign(self.weights[i])

            # Atualiza o gradiente com regularizações L1 e L2
            nabla_w[i] += l2_term + l1_term

            # Momentum: atualiza os termos de velocidade
            self.velocity_weights[i] = momentum * self.velocity_weights[i] - (eta / len(mini_batch)) * nabla_w[i]
            self.velocity_biases[i] = momentum * self.velocity_biases[i] - (eta / len(mini_batch)) * nabla_b[i]

            # Atualiza pesos e bias com os valores de velocidade
            self.weights[i] += self.velocity_weights[i]
            self.biases[i] += self.velocity_biases[i]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x
        activations = [x]
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation) + b
            nets.append(net)
            activation = relu(net)  # z = valor de saída do neurônio
            activations.append(activation)

        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * relu_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())  # (𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante.
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = relu_prime(net)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())  # ∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations - y)

# Função de Ativação ReLu
def relu(net):
    return np.maximum(0, net)

# Função para retornar as derivadas da função ReLu
def relu_prime(z):
    return np.where(z <= 0, 0, 1)

Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [44]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


DEFININDO AS ARQUITETURAS DOS 3 MELHORES RESULTADOS

melhores arquiteturas:

[784, 80, 40, 10] - lr = 0.3 - 93.50%

[784, 100, 10] - lr = 0.1 - 92.47%

[784, 64, 10] - lr = 0.3 - 90.35%

#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [ ]:
# Arquitetura da rede
#arquitecture = [784, 64, 10]
#arquitecture = [784, 100, 10]
arquitecture = [784, 80, 40, 10]

results = {
    "Configuracao": ["L1", "L2", "Momento"],
    "Acuracia": [0, 0, 0]
}

print("Implementacao da regularizacao L1:")
training_data_l1, validation_data_l1, test_data_l1 = load_data_wrapper()
mlp_l1 = Network(arquitecture)
results["Acuracia"][0] = mlp_l1.SGD(training_data_l1, 10, 32, 0.1, lambda_reg=0.001, momentum=0.0, test_data=test_data_l1)
print("-----------------------------------------------------------------------------\n")

print("Implementacao da regularizacao L2:")
training_data_l2, validation_data_l2, test_data_l2 = load_data_wrapper()
mlp_l2 = Network(arquitecture)
results["Acuracia"][1] = mlp_l2.SGD(training_data_l2, 10, 32, 0.1, lambda_reg=0.001, momentum=0.0, test_data=test_data_l2)
print("-----------------------------------------------------------------------------\n")

print("Implementacao do Momentum:")
training_data_momentum, validation_data_momentum, test_data_momentum = load_data_wrapper()
mlp_momentum = Network(arquitecture)
results["Acuracia"][2] = mlp_momentum.SGD(training_data_momentum, 10, 32, 0.1, lambda_reg=0.0, momentum=0.6, test_data=test_data_momentum)
print("-----------------------------------------------------------------------------")

print("\nTabela de Resultados:")
print("Configuracao | Acuracia")
print("-------------------------")
for config, acc in zip(results["Configuracao"], results["Acuracia"]):
    print(f"{config:<12} | {acc:.2f}%")


Implementacao da regularizacao L1:
Epoch 0 : 7665 / 10000 = 76.65%
Epoch 1 : 7739 / 10000 = 77.39%
Epoch 2 : 7839 / 10000 = 78.39%
Epoch 3 : 7844 / 10000 = 78.44%
Epoch 4 : 7880 / 10000 = 78.80%
Epoch 5 : 7887 / 10000 = 78.87%
Epoch 6 : 7895 / 10000 = 78.95%
Epoch 7 : 7906 / 10000 = 79.06%
Epoch 8 : 7898 / 10000 = 78.98%
Epoch 9 : 7909 / 10000 = 79.09%

Acurácia média após 10 épocas: 78.46%
-----------------------------------------------------------------------------

Implementacao da regularizacao L2:
Epoch 0 : 8510 / 10000 = 85.10%
Epoch 1 : 8614 / 10000 = 86.14%
Epoch 2 : 8682 / 10000 = 86.82%
Epoch 3 : 8714 / 10000 = 87.14%
Epoch 4 : 8717 / 10000 = 87.17%
Epoch 5 : 8756 / 10000 = 87.56%
Epoch 6 : 8771 / 10000 = 87.71%
Epoch 7 : 8783 / 10000 = 87.83%
Epoch 8 : 8776 / 10000 = 87.76%
Epoch 9 : 8785 / 10000 = 87.85%

Acurácia média após 10 épocas: 87.11%
-----------------------------------------------------------------------------

Implementacao do Momentum:
Epoch 0 : 7673 / 10000 = 76

##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:

1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porque de cada resultado? Qual foi a melhor regularização? Por quê? O Momento melhorou os resultados? Por quê?

R - Antes das regularizações, a melhor arquitetura foi [784, 80, 40, 10] com uma acurácia de 93.50% e uma taxa de aprendizado (lr) de 0.3.
Após as regularizações, a configuração [784, 64, 10] com L1 obteve uma acurácia de 96.55%, mostrando que a regularização L1 foi eficaz em reduzir o overfitting e melhorar a generalização. Porem, a L2 nessa mesma configuração teve um desempenho inferior (87.41%), sugerindo que a L1 foi mais adequada.
Na configuração [784, 100, 10], a L2 se destacou com uma acurácia de 96.84%, enquanto a L1 teve um desempenho menor (87.33%). Isso demonstra que a eficácia da regularização pode depender da arquitetura da rede. O uso de momentum na configuração [784, 64, 10] resultou em uma leve melhoria (96.96%) em comparação com a L1, indicando que o momentum pode acelerar a convergência. No entanto, na configuração [784, 100, 10], o momentum teve um desempenho inferior (86.08%) em relação à L2, sugerindo que sua eficácia pode variar conforme a arquitetura e a taxa de aprendizado.
Em resumo, a melhor regularização foi a L1 para a configuração [784, 64, 10], enquanto a L2 se destacou na configuração [784, 100, 10]. O momentum melhorou os resultados em algumas configurações, mas não em todas, indicando que sua eficácia depende da interação com a arquitetura da rede e os hiperparâmetros utilizados.


Data de Entrega: 14/10/2020
     
      
Complete a Tabela abaixo com os resultados

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & 1 & 2 & 3 \\ \hline
L1 & 96.55 & 87.33 & 78.46  \\ \hline
L2 & 87.41 & 96.84 & 87.11 \\ \hline
Momento  & 96.96& 86.08& 82.40\\ \hline
\end{array}



\\
Data de Entrega: 17/10/2020
